# Fine Tuning Schlager Bot

Using the dataset of colelcted Schlager song lyrics, we will fine tune a German based LLM to generate song lyrics from a provided verse.

### Formatting the Data into Dataset

In [8]:
#Import dependencies

from random import randrange
import pandas as pd

In [9]:
df_lyrics = pd.read_csv('schlager_songs.csv')

df_lyrics['lyrics'][10]

'{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}'

In [10]:
# Create separate column for 1st verse

verse_list = []

for ind in df_lyrics["lyrics"].index:
    lyrics = df_lyrics["lyrics"].iloc[ind]
    if lyrics == '{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}' :
        verse_list.append(None)
    else:
        try:
            verse = lyrics.split("\n")[0]
            n = 1
            while len(' '.join(verse).split()) <= 15:
                verse = lyrics.split("\n")[0:n]
                n += 1
            verse_list.append(''.join(verse))

        except:
            verse_list.append(None)


df_lyrics["verse_1"] = verse_list

df_lyrics["verse_1"][0]

df_lyrics.to_csv("schlager_songs_v2.csv")


In [11]:
# Prompt Example

### Instruction:

"Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben."

### Input:

"Du sagst nicht ein Wort"

### Response:

f""" Du sagst nicht ein Wort
 Und deine Hand wischt eine Träne fort
 Und dein leerer Blick
 Sinkt in dein Glas
 Du sitzt hier vor mir
 Und dein Gesicht lässt keinen Zweifel mehr
 Heut sagst du mir
 Dass ich dich verlier'
 Nie war Zeit für dich
 Ich lebte nur in meiner eignen Welt
 Ich weiß, du wirst gehen
 Ich muss dich versteh'n
 Lieb mich ein letztes mal
 Es bleibt mir keine andre Wahl
 Ich weiß, dass ich die Nacht mit dir
 An den Tag verlier'
...
 Und bleib bei mir"""

" Du sagst nicht ein Wort\n Und deine Hand wischt eine Träne fort\n Und dein leerer Blick\n Sinkt in dein Glas\n Du sitzt hier vor mir\n Und dein Gesicht lässt keinen Zweifel mehr\n Heut sagst du mir\n Dass ich dich verlier'\n Nie war Zeit für dich\n Ich lebte nur in meiner eignen Welt\n Ich weiß, du wirst gehen\n Ich muss dich versteh'n\n Lieb mich ein letztes mal\n Es bleibt mir keine andre Wahl\n Ich weiß, dass ich die Nacht mit dir\n An den Tag verlier'\n...\n Und bleib bei mir"

In [12]:
def format_instruction(sample):
    return f"""### Instruction:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

{sample['input']}

### Response:

{sample['output']}

"""

In [13]:
#Create input and output dataset from dataframe

from datasets import Dataset

df_dataset = df_lyrics[["verse_1", "lyrics"]]

ind_nan = []

for ind in df_dataset.index:
    if df_dataset["verse_1"].iloc[ind] == None:
        ind_nan.append(ind)
    else:
        continue

df_dataset.drop(index=ind_nan, inplace=True)

df_dataset.reset_index(drop=True, inplace=True)

df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)

dataset = Dataset.from_pandas(df_dataset)

dataset


ImportError: cannot import name 'DatasetCard' from 'huggingface_hub' (c:\Users\nicla\anaconda3\envs\kaggle\Lib\site-packages\huggingface_hub\__init__.py)

In [ ]:
print(fromat_instruction(dataset[randrange(len(dataset))]))

### Instruction:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

 Du sagst nicht ein Wort

### Response:

 Du sagst nicht ein Wort
 Und deine Hand wischt eine Träne fort
 Und dein leerer Blick
 Sinkt in dein Glas
 Du sitzt hier vor mir
 Und dein Gesicht lässt keinen Zweifel mehr
 Heut sagst du mir
 Dass ich dich verlier'
 Nie war Zeit für dich
 Ich lebte nur in meiner eignen Welt
 Ich weiß, du wirst gehen
 Ich muss dich versteh'n
 Lieb mich ein letztes mal
 Es bleibt mir keine andre Wahl
 Ich weiß, dass ich die Nacht mit dir
 An den Tag verlier'
 Schenk mir die Zeit
 Die uns noch bleibt
 Lieb mich ein letztes Mal
 Lass' mich dich noch einmal spür'n
 Bist du auch morgen nicht mehr hier
 Etwas bleibt von dir
 Wenn ich erwach'
 Alleine erwach'
 Ich hab' so viel Zeit
 Neben dir, an dir vorbei gelebt
 Ich nahm, doch was ich gab
 Zählte nicht viel
 Du hast oft geweint
 Doch ich war blind
 Sah deine Tränen nicht
 Heut ist mir klar
 Wie einsam du warst
 Lie

### Instruction-tune model using trl and the SFTTrainer

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model id
model_id = "philschmid/instruct-igel-001" # non-gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
#SFTTrainer supports a native integration with peft, which makes it easy to instruction tune LLMs

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
#We need to define the hyperparameters before we can start training

from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="igel_schlager_001",
    num_train_epochs=3,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=True # disable tqdm since with packing values are in correct
)

In [ ]:
#We can now configure the trainer

from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)


In [ ]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()

### Test Model and run Inference

In [ ]:
# Following training we want to run and test our model

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

args.output_dir = "igel_schlager_001"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

In [ ]:
# Load dataset with a random sample and try to generate the lyrics

#TODO relaod dataset and sample

sample = randrange(len(dataset))

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample['response']}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.9)

print(f"Prompt:\n{sample['response']}\n")
print(f"Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"Ground truth:\n{sample['instruction']}")

In [ ]:
import torch

print(torch._C._cuda_getCompiledVersion())

11080
